In [ ]:
# Importing Potentially Used Basic Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import sqlalchemy as db
from sqlalchemy import create_engine
import io
import psycopg2

In [ ]:
from config import db_password, username


In [ ]:
host = 'movies-fp.cpige012zhtw.us-east-1.rds.amazonaws.com'
port = 5432
passw = db_password
database = "postgres"
port=5432

   
db_string = "postgresql://" + username + ":" + passw + "@" + host + ":" + "5432/" + database
engine = db.create_engine(db_string)
connection = engine.connect()
metadata = db.MetaData()

In [ ]:
movie_genres = db.Table('movie_genre', metadata, autoload=True, autoload_with=engine)
query = db.select([movie_genres]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
mov_gen_df = pd.DataFrame(ResultSet)
mov_gen_df.columns = ResultSet[0].keys()
mov_gen_df = mov_gen_df.loc[:, ['movie_id', 'mg_id', 'genre_id']]

mov_gen_df.head()

In [ ]:
movies = db.Table('movies', metadata, autoload=True, autoload_with=engine)
query = db.select([movies]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
movies_df = pd.DataFrame(ResultSet)
movies_df.columns = ResultSet[0].keys()

movies_df.head()

In [ ]:
ratings = db.Table('ratings', metadata, autoload=True, autoload_with=engine)
query = db.select([ratings]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ratings_df = pd.DataFrame(ResultSet)
ratings_df.columns = ResultSet[0].keys()
ratings_df = ratings_df.loc[:, ['movie_id', 'user_id', 'rating']]

ratings_df.head()

In [ ]:
connection.close()

In [ ]:
movies_df = movies_df.loc[:, ["name", "year", "movie_id"]]
movies_df = movies_df.set_index("movie_id")
ratings_df = ratings_df.loc[:, ['movie_id', 'user_id', 'rating']]
ratings_df = ratings_df.set_index("movie_id")
mov_gen_df = mov_gen_df.loc[:, ['movie_id', 'mg_id', 'genre_id']]
mov_gen_df = mov_gen_df.set_index("movie_id")

In [ ]:
data_df = ratings_df.join(movies_df, on="movie_id", how="left", sort=True)
data_df = data_df.join(mov_gen_df, on="movie_id", how="left", sort=True)
data_df.head()

In [ ]:
data_df["movie_id"] = data_df.index
data_df.head()

In [ ]:
data_df = data_df.reset_index(drop=True)
data_df.head()

In [ ]:
data_df.count()

In [ ]:
data_df = data_df.dropna()

In [ ]:
data_df.count()

In [ ]:
data_df.to_csv("ML_Static_Dataframe.csv", index=False)